<a href="https://colab.research.google.com/github/shantanu2383/ST-MOMENTUM/blob/main/ST_MOMENTUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# IMPORT + CLEAN DATA

In [ ]:
filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"
file="CRSP_COMPUSTAT_MERGED.csv"
df=pd.read_csv(filepath + file)

filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"

file="CRSPMONTHLY.csv"
crsp=pd.read_csv(filepath + file)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
crsp['date'] = pd.to_datetime(crsp['date'], format="%Y%m%d")


crsp=crsp[("1963-01-01"<=crsp['date']) & (crsp['date']< "2019-01-01")]
crsp_parsed=crsp[['PERMNO', 'date', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD', 'VOL']]
crsp=crsp_parsed


x=['PERMNO', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD']

for i in x:
  crsp[i]=pd.to_numeric(crsp[i], errors='coerce')


#convert returns to percent:
crsp['RET']*=100
crsp['DLRET']*=100


#add market cap column
crsp['mkt_cap']=abs(crsp['SHROUT'] * crsp['ALTPRC'])/1000
crsp['mkt_cap'].replace(0, np.NaN, inplace=True)


#ONLY KEEP US BASED COMMON STOCKS
crsp=crsp[(crsp['SHRCD']==10)|(crsp['SHRCD']==11)]

#rename variables to lower case
crsp= crsp.rename(columns=str.lower)

#filter for relevant exchanges
exchange_mapping = {
    1: 'NYSE', 31: 'NYSE',
    2: 'AMEX', 32: 'AMEX',
    3: 'NASDAQ', 33: 'NASDAQ'
}
#create dictionary to map values of the 'exchcd' column



crsp['exchange'] = np.select(
    [crsp['exchcd'].isin(exchange_mapping.keys()), crsp['exchcd'].notnull()],
    [crsp['exchcd'].map(exchange_mapping), "Other"],
    default='Other')

#only keep NYSE, AMEX and NASDAQ stocks
crsp=crsp[(crsp['exchange'] == "NYSE") | (crsp['exchange'] == "AMEX") | (crsp['exchange'] == "NASDAQ")  ]


#Adjust Returns
crsp['ret_adj'] = crsp['ret']

mask1 = pd.isnull(crsp['dlstcd'])
crsp.loc[mask1, 'ret_adj'] = crsp.loc[mask1, 'ret']

mask2 = pd.notnull(crsp['dlstcd']) & pd.notnull(crsp['dlret'])
crsp.loc[mask2, 'ret_adj'] = crsp.loc[mask2, 'dlret']

mask3 = ((551 <= crsp['dlstcd']) & (crsp['dlstcd'] <= 574)) | (crsp['dlstcd'].isin([500, 520, 580, 584]))
crsp.loc[mask3, 'ret_adj'] = -30

crsp.loc[~(mask1 | mask2 | mask3), 'ret_adj'] = -100

crsp.drop(['shrcd', 'dlret', 'dlstcd'], inplace=True, axis=1)

#exclude financial firms 
crsp=crsp[(crsp['siccd']<6000) |(crsp['siccd']>6999) ]

#exclude stocks not in compustat

crsp_filtered=crsp[crsp['permno'].isin(df['LPERMNO'])]

#exclude stocks not in compustat

#crsp_filtered=crsp[crsp['permno'].isin(df['LPERMNO'])]

crsp['vol_adjusted'] = np.where((crsp['exchange'] == 'NASDAQ') & (crsp['date'] < '2001-02-01'), crsp['vol'] / 2,
                                 np.where((crsp['exchange'] == 'NASDAQ') & (crsp['date'] < '2002-01-01'), crsp['vol'] / 1.8,
                                          np.where((crsp['exchange'] == 'NASDAQ') & (crsp['date'] < '2004-01-01'), crsp['vol'] / 1.6,
                                                   crsp['vol'])))

crsp['date']=pd.to_datetime(crsp['date'])
crsp['year']=crsp['date'].dt.year

crsp['reference_date']=pd.to_datetime(crsp['year'].astype(str) +'-06-01')

crsp.loc[crsp['date'].dt.month < 6, 'year'] -= 1

crsp.loc[crsp['date'].dt.month < 6, 'reference_date'] = pd.to_datetime(
    crsp['year'].astype(str) + '-06-01')

crsp = crsp.drop('year', axis=1)

#extract market cap at June
crsp['month']=crsp['date'].dt.month
stock_weights=crsp[crsp['month']==6]


stock_weights.to_sql('weights', conn, if_exists='replace')
crsp.to_sql('crsp', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, weights.mkt_cap as mkt_equity
        FROM crsp
        LEFT JOIN weights ON crsp.reference_date=weights.reference_date AND crsp.permno=weights.permno
        '''

crsp_weights=pd.read_sql(query, conn)
crsp=crsp_weights

#deflate trading volume by shares outstanding:

crsp['to']=crsp['vol_adjusted']/crsp['shrout']

crsp=crsp.set_index('date')
crsp.sort_index(inplace=True)
crsp['return_t-1']=crsp.groupby('permno')['ret_adj'].shift(1)
crsp['to_t-1']=crsp.groupby('permno')['to'].shift(1)


<ipython-input-4-54661ab33de5>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['ret_adj'] = crsp['ret']


Independent Lagged Return Sorts

In [ ]:
nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date']).apply(lambda x: x.set_index('permno')['return_t-1'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.1: 'Q1', 0.2: 'Q2', 0.3:'Q3', 0.4:'Q4', 0.5:'Q5', 0.6:'Q6', 0.7:'Q7', 0.8:'Q8', 0.9: 'Q9'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse_breakpoints', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q1, nyse.Q2, nyse.Q3, nyse.Q4, nyse.Q5, nyse.Q6, nyse.Q7, nyse.Q8, nyse.Q9
        FROM crsp
        LEFT JOIN nyse_breakpoints nyse
        ON crsp.date=nyse.date
        '''

crsp_nyse_bp=pd.read_sql(query, conn)

crsp_nyse_bp['lagged_return_portfolio'] = np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q1'], 'P1',
                                  np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q2'], 'P2',
                                           np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q3'], 'P3',
                                                    np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q4'], 'P4',
                                                             np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q5'], 'P5',
                                                                      np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q6'], 'P6',
                                                                               np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q7'], 'P7',
                                                                                        np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q8'], 'P8',
                                                                                                 np.where(crsp_nyse_bp['return_t-1'] < crsp_nyse_bp['Q9'], 'P9', 'P10')
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )


crsp_nyse_bp
for x in [1,2,3,4,5,6,7,8,9]:
  crsp_nyse_bp.drop([f'Q{x}'], axis=1, inplace=True)

crsp=crsp_nyse_bp

Conditional Lagged Turnover Sorts

In [ ]:
nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date', 'lagged_return_portfolio']).apply(lambda x: x.set_index('permno')['to_t-1'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.1: 'Q1', 0.2: 'Q2', 0.3:'Q3', 0.4:'Q4', 0.5:'Q5', 0.6:'Q6', 0.7:'Q7', 0.8:'Q8', 0.9: 'Q9'}, inplace=True)

crsp['date'] = pd.to_datetime(crsp['date'])
nyse_monthly['date'] = pd.to_datetime(nyse_monthly['date'])

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q1, nyse.Q2, nyse.Q3, nyse.Q4, nyse.Q5, nyse.Q6, nyse.Q7, nyse.Q8, nyse.Q9
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date AND crsp.lagged_return_portfolio=nyse.lagged_return_portfolio
        '''

crsp_nyse_bp=pd.read_sql(query, conn)

crsp_nyse_bp['lagged_turnover_portfolio'] = np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q1'], 'P1',
                                  np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q2'], 'P2',
                                           np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q3'], 'P3',
                                                    np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q4'], 'P4',
                                                             np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q5'], 'P5',
                                                                      np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q6'], 'P6',
                                                                               np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q7'], 'P7',
                                                                                        np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q8'], 'P8',
                                                                                                 np.where(crsp_nyse_bp['to_t-1'] < crsp_nyse_bp['Q9'], 'P9', 'P10')
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )

crsp_nyse_bp.dropna(inplace=True)
for x in [1,2,3,4,5,6,7,8,9]:
  crsp_nyse_bp.drop([f'Q{x}'], axis=1, inplace=True)

crsp=crsp_nyse_bp

Export Clean CRSP File

In [ ]:
crsp.to_csv(filepath + "cleaned_crsp.csv")

Import Fama French Data

In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
datasets = get_available_datasets()
#FF 5 FACTORS + MOMENTUM FACTOR
df_5_factor=[dataset for dataset in datasets if 'Research' in dataset and 'Factor' in dataset]
df_mom_factor=[dataset for dataset in datasets if 'Momentum' in dataset and 'Factor' in dataset]

ff=web.DataReader(df_5_factor[3],'famafrench',start='1963-07-01',end='2022-11-01')[0]
ff_mom=web.DataReader(df_mom_factor[0],'famafrench',start='1963-07-01',end='2022-11-01')[0]

ff=pd.merge(ff, ff_mom, on='Date')
ff.reset_index(inplace=True)
ff.rename(columns={'Date':'month_year'}, inplace=True)
ff['month_year']=ff['month_year'].apply(str)


#FF ST AND LT REVERSAL FACTORS
df_ST_LT_factor=[dataset for dataset in datasets if 'ST_Reversal' in dataset or 'LT_Reversal' in dataset]
df_ST_LT_factor
ff_ST=web.DataReader(df_ST_LT_factor[0],'famafrench',start='1963-07-01',end='2022-11-01')[0]
ff_LT=web.DataReader(df_ST_LT_factor[2],'famafrench',start='1963-07-01',end='2022-11-01')[0]

for x in [ff_ST, ff_LT]:
  x.reset_index(inplace=True)
  x.rename(columns={'Date': 'month_year'}, inplace=True)
  x['month_year']=x['month_year'].apply(str)

#merge ST REV AND LT REV TOGETHER
ff_rev=pd.merge(ff_ST, ff_LT, on='month_year')
ff_rev
#merge into main ff factors
ff=pd.merge(ff, ff_rev, on='month_year')


Import Zhang (2015) Q Factors

In [ ]:
filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"

q=pd.read_csv(filepath + "q5_factors_monthly_2022.csv")
q['month_year'] = pd.to_datetime(q['year'].astype(str) + '-' + q['month'].astype(str)).dt.to_period('M')
q.drop(['year', 'month'], axis=1, inplace=True)
q['month_year']=q['month_year'].apply(str)

Import Pastor and Stambaugh (2003) Traded Liquidity Factor (PSLIQ)

In [ ]:
psliq=pd.read_csv(filepath + "PSLIQ.csv")
psliq['DATE'] = pd.to_datetime(psliq['DATE'], format="%Y%m%d")
psliq['month_year']=psliq['DATE'].dt.to_period("M")
psliq.drop(['DATE', 'PS_INNOV', 'PS_LEVEL'], axis=1, inplace=True)
psliq['month_year']=psliq['month_year'].apply(str)

factors=pd.merge(ff, psliq, on='month_year')
factors.rename(columns={'PS_VWF': 'psliq'}, inplace=True)

#TABLE 1

In [ ]:
crsp=pd.read_csv(filepath + "cleaned_crsp.csv" )
#Winsorise returns at the 99% quantile
crsp['ret_adj'] = crsp['ret_adj'].clip(upper=crsp['ret_adj'].quantile(0.99))

import seaborn as sns
sns.boxplot(y='ret_adj', data=crsp)

In [ ]:

portfolio_returns = crsp.groupby(['date', 'lagged_return_portfolio', 'lagged_turnover_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret_adj']), weights=pd.to_numeric(x['mkt_equity']))).reset_index(name='weighted_returns')
portfolio_returns.sort_values(by=['date', 'lagged_return_portfolio', 'lagged_turnover_portfolio'], inplace=True)

portfolio_returns_ts = portfolio_returns.groupby(['lagged_return_portfolio', 'lagged_turnover_portfolio']).mean()


portfolio_returns_ts.reset_index(inplace=True)
portfolio_returns_ts=portfolio_returns_ts[['lagged_return_portfolio', 'lagged_turnover_portfolio', 'weighted_returns']]

portfolio_returns_ts_wide=pd.pivot(portfolio_returns_ts, index='lagged_turnover_portfolio', columns='lagged_return_portfolio', values='weighted_returns')
table1=portfolio_returns_ts_wide
table1

# SUB TABLE 1

For the strategy that goes long on the securities in the top turnover decile and shorts those in the lowest turnover decile, within each lagged return portfolio 

In [ ]:
portfolio_returns['date']=pd.to_datetime(portfolio_returns['date'])
portfolio_returns['month_year']=portfolio_returns['date'].dt.to_period("M")
portfolio_returns['month_year']=portfolio_returns['month_year'].apply(str)

Regress on FF6 Factors:

In [ ]:
intercepts=[]
t=[]

for x in ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"]:
  portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_return_portfolio']==x) & (portfolio_returns['lagged_turnover_portfolio']=="P1")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_return_portfolio']==x) & (portfolio_returns['lagged_turnover_portfolio']=="P10")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, ff, on='month_year')
  
  import statsmodels.api as sm
  
  #sns.boxplot(y='high-low', data=portfolio_returns_temp)
  plt.show()
  q_99 = portfolio_returns_temp['high-low'].quantile(0.99)
  q_01 = portfolio_returns_temp['high-low'].quantile(0.01)
  
  # Clip the 'high-low' column at the 99% and 1% quantiles
  portfolio_returns_temp['high-low'] = portfolio_returns_temp['high-low'].clip(lower=q_01, upper=q_99)
  # Define the dependent and independent variables
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']

  intercepts.append(intercept) 
  t.append(tvalue)

intercepts_df = pd.DataFrame(intercepts).transpose()
intercepts_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
intercepts_df.insert(0, "Turnover Strategies", "alpha_FF")
t_df = pd.DataFrame(t).transpose()
t_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
t_df.insert(0, "Turnover Strategies", "t_FF")

ff_t1= pd.concat([intercepts_df, t_df], axis=0)
ff_t1

Regress on Q factors

In [ ]:
intercepts=[]
t=[]

for x in ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"]:
  portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_return_portfolio']==x) & (portfolio_returns['lagged_turnover_portfolio']=="P1")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_return_portfolio']==x) & (portfolio_returns['lagged_turnover_portfolio']=="P10")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, q, on='month_year')
  
  import statsmodels.api as sm
  
  #sns.boxplot(y='high-low', data=portfolio_returns_temp)
  plt.show()
  q_99 = portfolio_returns_temp['high-low'].quantile(0.99)
  q_01 = portfolio_returns_temp['high-low'].quantile(0.01)
  
  # Clip the 'high-low' column at the 99% and 1% quantiles
  #portfolio_returns_temp['high-low'] = portfolio_returns_temp['high-low'].clip(lower=q_01, upper=q_99)
  # Define the dependent and independent variables
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']

  intercepts.append(intercept) 
  t.append(tvalue)

intercepts_df = pd.DataFrame(intercepts).transpose()
intercepts_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
intercepts_df.insert(0, "Turnover Strategies", "alpha_Q")

t_df = pd.DataFrame(t).transpose()
t_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
t_df.insert(0, "Turnover Strategies", "t_Q")

q_t1= pd.concat([intercepts_df, t_df], axis=0)
q_t1

Calculate the Expected Return

In [ ]:
averages=[]
t=[]
for x in ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"]:
  portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_return_portfolio']==x) & (portfolio_returns['lagged_turnover_portfolio']=="P1")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_return_portfolio']==x) & (portfolio_returns['lagged_turnover_portfolio']=="P10")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  avg=portfolio_returns_temp['high-low'].mean()
  s = portfolio_returns_temp['high-low'].std(ddof=1)
  n = len(portfolio_returns_temp)
  se = s / np.sqrt(n)
  t_stat = (avg - 0) / se

  averages.append(avg)
  t.append(t_stat)

averages_df= pd.DataFrame(averages).transpose()
averages_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
averages_df.insert(0, "Turnover Strategies", "E[R]")

t_df = pd.DataFrame(t).transpose()
t_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
t_df.insert(0, "Turnover Strategies", "t_E[R]")

r_t1= pd.concat([averages_df, t_df], axis=0)


In [ ]:
sub_table1_turnover=pd.concat([r_t1, ff_t1, q_t1], axis=0)
sub_table1_turnover.set_index("Turnover Strategies", inplace=True)
sub_table1_turnover

WE NOW REPEAT THE PROCESS FOR THE STRATEGY THAT GOES LONG ON THE WINNERS AND SHORTS THE LOSERS WITHIN EACH TURNOVER PORTFOLIO

In [ ]:
intercepts=[]
t=[]

for x in ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"]:
  portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']==x) & (portfolio_returns['lagged_return_portfolio']=="P1")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']==x) & (portfolio_returns['lagged_return_portfolio']=="P10")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, ff, on='month_year')
  
  import statsmodels.api as sm
  
  #sns.boxplot(y='high-low', data=portfolio_returns_temp)
  plt.show()
  q_99 = portfolio_returns_temp['high-low'].quantile(0.99)
  q_01 = portfolio_returns_temp['high-low'].quantile(0.01)
  
  #Clip the 'high-low' column at the 99% and 1% quantiles
  #portfolio_returns_temp['high-low'] = portfolio_returns_temp['high-low'].clip(lower=q_01, upper=q_99)
  # Define the dependent and independent variables
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']

  intercepts.append(intercept) 
  t.append(tvalue)

intercepts_df = pd.DataFrame(intercepts).transpose()
intercepts_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
intercepts_df.insert(0, "Lagged Return Strategies", "alpha_FF")
t_df = pd.DataFrame(t).transpose()
t_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
t_df.insert(0, "Lagged Return Strategies", "t_FF")

ff_t1= pd.concat([intercepts_df, t_df], axis=0)
ff_t1

In [ ]:
intercepts=[]
t=[]

for x in ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"]:
  portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']==x) & (portfolio_returns['lagged_return_portfolio']=="P1")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']==x) & (portfolio_returns['lagged_return_portfolio']=="P10")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, q, on='month_year')
  
  import statsmodels.api as sm
  
  #sns.boxplot(y='high-low', data=portfolio_returns_temp)
  plt.show()
  q_99 = portfolio_returns_temp['high-low'].quantile(0.99)
  q_01 = portfolio_returns_temp['high-low'].quantile(0.01)
  
  # Clip the 'high-low' column at the 99% and 1% quantiles
  #portfolio_returns_temp['high-low'] = portfolio_returns_temp['high-low'].clip(lower=q_01, upper=q_99)
  # Define the dependent and independent variables
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']

  intercepts.append(intercept) 
  t.append(tvalue)

intercepts_df = pd.DataFrame(intercepts).transpose()
intercepts_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
intercepts_df.insert(0, "Lagged Return Strategies", "alpha_Q")

t_df = pd.DataFrame(t).transpose()
t_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
t_df.insert(0, "Lagged Return Strategies", "t_Q")

q_t1= pd.concat([intercepts_df, t_df], axis=0)
q_t1

In [ ]:
averages=[]
t=[]
for x in ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"]:
  portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']==x) & (portfolio_returns['lagged_return_portfolio']=="P1")]

  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']==x) & (portfolio_returns['lagged_return_portfolio']=="P10")]

  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  avg=portfolio_returns_temp['high-low'].mean()
  s = portfolio_returns_temp['high-low'].std(ddof=1)
  n = len(portfolio_returns_temp)
  se = s / np.sqrt(n)
  t_stat = (avg - 0) / se

  averages.append(avg)
  t.append(t_stat)

averages_df= pd.DataFrame(averages).transpose()
averages_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
averages_df.insert(0, "Lagged Return Strategies", "E[R]")

t_df = pd.DataFrame(t).transpose()
t_df.columns = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"] 
t_df.insert(0, "Lagged Return Strategies", "t_E[R]")

r_t1= pd.concat([averages_df, t_df], axis=0)

In [ ]:
sub_table1_return=pd.concat([r_t1, ff_t1, q_t1], axis=0)
sub_table1_return.set_index("Lagged Return Strategies", inplace=True)
sub_table1_return

# Table 2

Factor Exposures and Abnormal Returns

In [ ]:
#Build STREV DATAFRAME- THIS IS THE STRATEGY THAT BUYS THE PREVIOUS MONTHS
# WINNERS AND SHORTS THE LOSERS WITHIN THE LOWEST TURNOVER DECILE

portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P1") & (portfolio_returns['lagged_return_portfolio']=="P1")]
portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P1") & (portfolio_returns['lagged_return_portfolio']=="P10")]
portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
st_rev=portfolio_returns_temp[['date', 'month_year', 'high-low']]
st_rev=pd.merge(st_rev, factors, on='month_year')

#Build STMOM DATAFRAME- THIS IS THE STRATEGY THAT BUYS THE PREVIOUS MONTHS
# WINNERS AND SHORTS THE LOSERS WITHIN THE HIGHEST TURNOVER DECILE

portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P10") & (portfolio_returns['lagged_return_portfolio']=="P1")]
portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P10") & (portfolio_returns['lagged_return_portfolio']=="P10")]
portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
st_mom=portfolio_returns_temp[['date', 'month_year', 'high-low']]
st_mom=pd.merge(st_mom, factors, on='month_year')

ST REV EXPOSURES:

In [ ]:
# create a dictionary with the column names and an empty list for each column
data = {'Independent Variable': ['Intercept', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom   ', 'ST_Rev', 'LT_Rev', 'PSLIQ', 'Adj. R2'],
        '(1)': ['', '', '', '', '', '', '', '', '', '', ''],
        '(2))': ['', '', '', '', '', '', '', '', '', '', ''],
        '(3))': ['', '', '', '', '', '', '', '', '', '', ''],
        '(4))': ['', '', '', '', '', '', '', '', '', '', '']}
df = pd.DataFrame(data=data, columns=['Independent Variable', '(1)', '(2)', '(3)', '(4)'])
df.set_index("Independent Variable", inplace=True)

y = st_rev['high-low']
X = st_rev[[]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 1 spec:", results.summary())
df.loc['Intercept', '(1)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"



y = st_rev['high-low']
X = st_rev[['Mkt-RF']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 2 spec:", results.summary())
df.loc['Intercept', '(2)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"
df.loc['Mkt-RF', '(2)'] = f"{results.params[1]:.2f} ({results.tvalues.loc['Mkt-RF']:.2f})"
df.loc['Adj. R2', '(2)'] = results.rsquared_adj*100

y = st_rev['high-low']
X = st_rev[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 3 spec:", results.summary())
df.loc['Intercept', '(3)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"
df.loc['Mkt-RF', '(3)'] = f"{results.params[1]:.2f} ({results.tvalues.loc['Mkt-RF']:.2f})"
df.loc['SMB', '(3)'] = f"{results.params[2]:.2f} ({results.tvalues.loc['SMB']:.2f})"
df.loc['HML', '(3)'] = f"{results.params[3]:.2f} ({results.tvalues.loc['HML']:.2f})"
df.loc['RMW', '(3)'] = f"{results.params[4]:.2f} ({results.tvalues.loc['RMW']:.2f})"
df.loc['CMA', '(3)'] = f"{results.params[5]:.2f} ({results.tvalues.loc['CMA']:.2f})"
df.loc['Mom   ', '(3)'] = f"{results.params[6]:.2f} ({results.tvalues.loc['Mom   ']:.2f})"
df.loc['Adj. R2', '(3)'] = results.rsquared_adj*100

y = st_rev['high-low']
X = st_rev[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ', 'ST_Rev', 'LT_Rev', 'psliq' ]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 4 spec:", results.summary())

df.loc['Intercept', '(4)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"
df.loc['Mkt-RF', '(4)'] = f"{results.params[1]:.2f} ({results.tvalues.loc['Mkt-RF']:.2f})"
df.loc['SMB', '(4)'] = f"{results.params[2]:.2f} ({results.tvalues.loc['SMB']:.2f})"
df.loc['HML', '(4)'] = f"{results.params[3]:.2f} ({results.tvalues.loc['HML']:.2f})"
df.loc['RMW', '(4)'] = f"{results.params[4]:.2f} ({results.tvalues.loc['RMW']:.2f})"
df.loc['CMA', '(4)'] = f"{results.params[5]:.2f} ({results.tvalues.loc['CMA']:.2f})"
df.loc['Mom   ', '(4)'] = f"{results.params[6]:.2f} ({results.tvalues.loc['Mom   ']:.2f})"
df.loc['ST_Rev', '(4)'] = f"{results.params[7]:.2f} ({results.tvalues.loc['ST_Rev']:.2f})"
df.loc['LT_Rev', '(4)'] = f"{results.params[8]:.2f} ({results.tvalues.loc['LT_Rev']:.2f})"
df.loc['PSLIQ', '(4)'] = f"{results.params[9]:.2f} ({results.tvalues.loc['psliq']:.2f})"
df.loc['Adj. R2', '(4)'] = results.rsquared_adj*100

st_rev=df

ST MOM EXPOSURES:

In [ ]:
# create a dictionary with the column names and an empty list for each column
data = {'Independent Variable': ['Intercept', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom   ', 'ST_Rev', 'LT_Rev', 'PSLIQ', 'Adj. R2'],
        '(1)': ['', '', '', '', '', '', '', '', '', '', ''],
        '(2))': ['', '', '', '', '', '', '', '', '', '', ''],
        '(3))': ['', '', '', '', '', '', '', '', '', '', ''],
        '(4))': ['', '', '', '', '', '', '', '', '', '', '']}
df = pd.DataFrame(data=data, columns=['Independent Variable', '(1)', '(2)', '(3)', '(4)'])
df.set_index("Independent Variable", inplace=True)


y = st_mom['high-low']
X = st_mom[[]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 1 spec:", results.summary())
df.loc['Intercept', '(1)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"

y = st_mom['high-low']
X = st_mom[['Mkt-RF']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 2 spec:", results.summary())
df.loc['Intercept', '(2)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"
df.loc['Mkt-RF', '(2)'] = f"{results.params[1]:.2f} ({results.tvalues.loc['Mkt-RF']:.2f})"
df.loc['Adj. R2', '(2)'] = results.rsquared_adj*100

y = st_mom['high-low']
X = st_mom[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 3 spec:", results.summary())
df.loc['Intercept', '(3)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"
df.loc['Mkt-RF', '(3)'] = f"{results.params[1]:.2f} ({results.tvalues.loc['Mkt-RF']:.2f})"
df.loc['SMB', '(3)'] = f"{results.params[2]:.2f} ({results.tvalues.loc['SMB']:.2f})"
df.loc['HML', '(3)'] = f"{results.params[3]:.2f} ({results.tvalues.loc['HML']:.2f})"
df.loc['RMW', '(3)'] = f"{results.params[4]:.2f} ({results.tvalues.loc['RMW']:.2f})"
df.loc['CMA', '(3)'] = f"{results.params[5]:.2f} ({results.tvalues.loc['CMA']:.2f})"
df.loc['Mom   ', '(3)'] = f"{results.params[6]:.2f} ({results.tvalues.loc['Mom   ']:.2f})"
df.loc['Adj. R2', '(3)'] = results.rsquared_adj*100


y = st_mom['high-low']
X = st_mom[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ', 'ST_Rev', 'LT_Rev', 'psliq' ]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
#print("model 4 spec:", results.summary())
df.loc['Intercept', '(4)'] = f"{results.params[0]:.2f} ({results.tvalues.loc['const']:.2f})"
df.loc['Mkt-RF', '(4)'] = f"{results.params[1]:.2f} ({results.tvalues.loc['Mkt-RF']:.2f})"
df.loc['SMB', '(4)'] = f"{results.params[2]:.2f} ({results.tvalues.loc['SMB']:.2f})"
df.loc['HML', '(4)'] = f"{results.params[3]:.2f} ({results.tvalues.loc['HML']:.2f})"
df.loc['RMW', '(4)'] = f"{results.params[4]:.2f} ({results.tvalues.loc['RMW']:.2f})"
df.loc['CMA', '(4)'] = f"{results.params[5]:.2f} ({results.tvalues.loc['CMA']:.2f})"
df.loc['Mom   ', '(4)'] = f"{results.params[6]:.2f} ({results.tvalues.loc['Mom   ']:.2f})"
df.loc['ST_Rev', '(4)'] = f"{results.params[7]:.2f} ({results.tvalues.loc['ST_Rev']:.2f})"
df.loc['LT_Rev', '(4)'] = f"{results.params[8]:.2f} ({results.tvalues.loc['LT_Rev']:.2f})"
df.loc['PSLIQ', '(4)'] = f"{results.params[9]:.2f} ({results.tvalues.loc['psliq']:.2f})"
df.loc['Adj. R2', '(4)'] = results.rsquared_adj*100
st_mom=df

Clean Table 2

In [ ]:
st_mom = pd.concat([pd.DataFrame([["ST_MOM "]*4,], columns=st_mom.columns, index=['Strategy']), st_mom])
st_rev = pd.concat([pd.DataFrame([["ST_REV "]*4,], columns=st_rev.columns, index=['Strategy']), st_rev])
st_rev.rename(columns={"(1)": "(5)", "(2)": "(6)", "(3)": "(7)", "(4)": "(8)"}, inplace=True)
table2=st_mom.join(st_rev)

In [ ]:
table2

# Figure 3

In [ ]:
crsp=pd.read_csv(filepath + "cleaned_crsp.csv" )
#Winsorise returns at the 99% quantile
#crsp['ret_adj'] = crsp['ret_adj'].clip(upper=crsp['ret_adj'].quantile(0.99))

import seaborn as sns
sns.boxplot(y='ret_adj', data=crsp)

In [ ]:
crsp
for i in range(1,25):
  crsp[f'return_t+{i}']=crsp.groupby('permno')['ret_adj'].shift(-i)


crsp
for i in range(1,25):
  crsp[f'me_t+{i}']=crsp.groupby('permno')['mkt_equity'].shift(-i)



crsp=crsp[(crsp['lagged_return_portfolio']=="P1") | (crsp['lagged_turnover_portfolio']=="P1")| (crsp['lagged_turnover_portfolio']=="P10")| (crsp['lagged_return_portfolio']=="P10") ]

cols = [f"return_t+{i}" for i in range(1, 25)]

weights_dict = {}
for i, column in enumerate(cols):
    weights_dict[column] = f"me_t+{i+1}"

crsp.dropna(inplace=True)

portfolio_returns = crsp.groupby(['date', 'lagged_return_portfolio', 'lagged_turnover_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret_adj']), weights=pd.to_numeric(x['mkt_equity']))).reset_index(name='weighted_returns')

for col in cols:
    weight_col = weights_dict[col]
    weighted_avg = crsp.groupby(['date', 'lagged_return_portfolio', 'lagged_turnover_portfolio']).apply(lambda x: np.average(pd.to_numeric(x[col]), weights=pd.to_numeric(x[weight_col]))).reset_index(name='weighted_' + col)
    portfolio_returns = portfolio_returns.merge(weighted_avg, on=['date', 'lagged_return_portfolio', 'lagged_turnover_portfolio'], how='left')




In [ ]:
#Build STMOM DATAFRAME- THIS IS THE STRATEGY THAT BUYS THE PREVIOUS MONTHS
# WINNERS AND SHORTS THE LOSERS WITHIN THE HIGHEST TURNOVER DECILE

portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P10") & (portfolio_returns['lagged_return_portfolio']=="P1")]
#portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P10") & (portfolio_returns['lagged_return_portfolio']=="P10")]
#portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')

portfolio_returns_temp

portfolio_returns_temp

for c in range(1,25):
  portfolio_returns_temp[f'high-low_{c}']=portfolio_returns_temp[f'weighted_return_t+{c}_y']-portfolio_returns_temp[f'weighted_return_t+{c}_x']
portfolio_returns_temp
for i in range (2, 25):
  portfolio_returns_temp[f'high-low_{i}']=portfolio_returns_temp[f'high-low_{i}'] + portfolio_returns_temp[f'high-low_{i-1}']

cols = [f"high-low_{i}" for i in range(1, 25)]
g=portfolio_returns_temp[cols].mean()
std = portfolio_returns_temp[cols].std()

g = pd.concat([g, std], axis=1)
g.columns = ['mean', 'std']


#g['cumulative_sum'] = g[0].cumsum()

import numpy as np
g['std'] = g['std']/np.sqrt(len(portfolio_returns_temp))

import scipy.stats as stats


g['upper_band']=g['mean']+(1.96*g['std'])
g['lower_band']=g['mean']-(1.96*g['std'])



In [ ]:
#Build STMOM DATAFRAME- THIS IS THE STRATEGY THAT BUYS THE PREVIOUS MONTHS
# WINNERS AND SHORTS THE LOSERS WITHIN THE HIGHEST TURNOVER DECILE

portfolio_returns_temp1=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P1") & (portfolio_returns['lagged_return_portfolio']=="P1")]
#portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

portfolio_returns_temp10=portfolio_returns[(portfolio_returns['lagged_turnover_portfolio']=="P1") & (portfolio_returns['lagged_return_portfolio']=="P10")]
#portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)
  
portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')

portfolio_returns_temp

portfolio_returns_temp

for c in range(1,25):
  portfolio_returns_temp[f'high-low_{c}']=portfolio_returns_temp[f'weighted_return_t+{c}_y']-portfolio_returns_temp[f'weighted_return_t+{c}_x']
for i in range (2, 25):
  portfolio_returns_temp[f'high-low_{i}']=portfolio_returns_temp[f'high-low_{i}'] + portfolio_returns_temp[f'high-low_{i-1}']


cols = [f"high-low_{i}" for i in range(1, 25)]

g_1=portfolio_returns_temp[cols].mean()
std = portfolio_returns_temp[cols].std()

g_1 = pd.concat([g_1, std], axis=1)
g_1.columns = ['mean', 'std']

import numpy as np
g_1['std'] = g_1['std']/np.sqrt(len(portfolio_returns_temp))


g_1['upper_band']=g_1['mean']+(1.96*g_1['std'])
g_1['lower_band']=g_1['mean']-(1.96*g_1['std'])



In [ ]:

# Plot first set of data
plt.plot(g.index, g['mean'], color='black', linestyle='solid', label='ST_MOM')

# Add upper and lower band lines for first set of data
plt.plot(g.index, g['upper_band'], color='black', linestyle='dashed')
plt.plot(g.index, g['lower_band'], color='black', linestyle='dashed')

# Plot second set of data
plt.plot(g_1.index, g_1['mean'], color='red', linestyle='solid', label='ST REV')

# Add upper and lower band lines for second set of data
plt.plot(g_1.index, g_1['upper_band'], color='red', linestyle='dashed')
plt.plot(g_1.index, g_1['lower_band'], color='red', linestyle='dashed')

# Set y-axis limits
plt.ylim(-3, 8)

# Add horizontal line at y=0
plt.axhline(y=0, color='grey')

# Add legend
plt.legend()
# Set x-axis tickers
plt.xticks([0, 5, 11, 17, 23], ['1', '6', '12','18', '24'])
plt.grid(axis='both')
plt.grid(color='gray', linestyle=':', linewidth='0.5')



# Show the plot
plt.show()

#Table 3 

# Panel A

In [ ]:
filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"

crsp=pd.read_csv(filepath + "cleaned_crsp.csv" )
crsp['date']=pd.to_datetime(crsp['date'])
crsp.drop(['lagged_return_portfolio', 'lagged_turnover_portfolio'], axis=1, inplace=True)
#shift back mkt_cap
crsp['mkt_cap_t-1']=crsp.groupby('permno')['mkt_cap'].shift(1)
crsp.drop(['Unnamed: 0', 'level_0'], axis=1, inplace=True)

Sort Bases on NYSE Median Size Breakpoints

In [ ]:
#Independent NYSE Sorts
nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date']).apply(lambda x: x.set_index('permno')['mkt_cap_t-1'].quantile([0.5]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.5: 'median'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.median
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date
        '''
crsp_nyse_size_bp=pd.read_sql(query, conn)

crsp_nyse_size_bp.dropna(inplace=True)
crsp=crsp_nyse_size_bp
crsp['size_portfolio']=np.where(crsp['mkt_cap_t-1']>crsp['median'], 'Large Cap', 'Small Cap')
crsp.drop(['median'], axis=1, inplace=True)

Conditional Lagged Return Sorts

In [ ]:
crsp.drop('level_0', axis=1, inplace=True)

nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date', 'size_portfolio']).apply(lambda x: x.set_index('permno')['return_t-1'].quantile([0.2, 0.8]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.2: 'Q2', 0.8: 'Q8'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q2, nyse.Q8
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date AND crsp.size_portfolio=nyse.size_portfolio
        '''
crsp_nyse_ret_bp=pd.read_sql(query, conn)

crsp_nyse_ret_bp.dropna(inplace=True)

crsp=crsp_nyse_ret_bp

crsp['lagged_return_portfolio'] = np.where(crsp['return_t-1'] < crsp['Q2'], 'Low Return', 
                                            np.where((crsp['Q2'] <= crsp['return_t-1']) & 
                                                     (crsp['return_t-1'] <= crsp['Q8']), 'Medium Return', 
                                                     'High Return'))
crsp.drop(['Q2', 'Q8'], axis=1, inplace=True)

Conditional Turnover Sorts

In [ ]:
crsp.drop('level_0', axis=1, inplace=True)

nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date', 'size_portfolio', 'lagged_return_portfolio']).apply(lambda x: x.set_index('permno')['to_t-1'].quantile([0.2, 0.8]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.2: 'Q2', 0.8: 'Q8'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q2, nyse.Q8
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date AND crsp.size_portfolio=nyse.size_portfolio AND crsp.lagged_return_portfolio=nyse.lagged_return_portfolio
        '''
crsp_nyse_to_bp=pd.read_sql(query, conn)

crsp=crsp_nyse_to_bp

crsp['lagged_turnover_portfolio'] = np.where(crsp['to_t-1'] < crsp['Q2'], 'Low Turnover', 
                                            np.where((crsp['Q2'] <= crsp['to_t-1']) & 
                                                     (crsp['to_t-1'] <= crsp['Q8']), 'Medium Turnover', 
                                                     'High Turnover'))

crsp.drop(['Q2', 'Q8'], axis=1, inplace=True)

Export Data:

In [ ]:
crsp.to_csv(filepath+"crsp_nyse_median_sorts.csv")

In [ ]:
portfolio_returns = crsp.groupby(['date', 'size_portfolio', 'lagged_return_portfolio', 'lagged_turnover_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret_adj']), weights=pd.to_numeric(x['mkt_equity']))).reset_index(name='weighted_returns')
portfolio_returns.sort_values(by=['date','size_portfolio', 'lagged_return_portfolio', 'lagged_turnover_portfolio'], inplace=True)

In [ ]:
import statsmodels.api as sm

#ST MOMENTUM(WML HIGH TURNOVER)
df = pd.DataFrame(columns=['E[R]', 'alpha_FF6', 'alpha_q'])
df.loc['Small Cap'] = [None, None, None]
df.loc['Large Cap'] = [None, None, None]

for x in ['Small Cap', 'Large Cap']:
  portfolio_returns['date']=pd.to_datetime(portfolio_returns['date'])
  portfolio_returns['month_year']=portfolio_returns['date'].dt.to_period("M")
  portfolio_returns['month_year']=portfolio_returns['month_year'].apply(str)

  portfolio_returns_size=portfolio_returns[portfolio_returns['size_portfolio']==x]

  portfolio_returns_temp1=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='High Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="Low Return")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='High Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="High Return")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)


  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, ff, on='month_year')

  
  #ff regression
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_FF6'] =f"{intercept:.2f} ({tvalue:.2f})"

  #q regression
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, q, on='month_year')
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]
  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_q'] =f"{intercept:.2f} ({tvalue:.2f})"

  
  #expected return
  avg=portfolio_returns_temp['high-low'].mean()
  s = portfolio_returns_temp['high-low'].std(ddof=1)
  n = len(portfolio_returns_temp)
  se = s / np.sqrt(n)
  t_stat = (avg - 0) / se
  
  df.loc[x, 'E[R]'] =f"{avg:.2f} ({t_stat:.2f})"

st_mom_A=df

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [ ]:
#ST REV
df = pd.DataFrame(columns=['E[R]', 'alpha_FF6', 'alpha_q'])
df.loc['Small Cap'] = [None, None, None]
df.loc['Large Cap'] = [None, None, None]

for x in ['Small Cap', 'Large Cap']:
  portfolio_returns['date']=pd.to_datetime(portfolio_returns['date'])
  portfolio_returns['month_year']=portfolio_returns['date'].dt.to_period("M")
  portfolio_returns['month_year']=portfolio_returns['month_year'].apply(str)

  portfolio_returns_size=portfolio_returns[portfolio_returns['size_portfolio']==x]

  portfolio_returns_temp1=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='Low Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="Low Return")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='Low Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="High Return")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)


  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, ff, on='month_year')

  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_FF6'] =f"{intercept:.2f} ({tvalue:.2f})"

  print(intercept)
  print(tvalue)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, q, on='month_year')
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]
  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_q'] =f"{intercept:.2f} ({tvalue:.2f})"

  

  avg=portfolio_returns_temp['high-low'].mean()
  s = portfolio_returns_temp['high-low'].std(ddof=1)
  n = len(portfolio_returns_temp)
  se = s / np.sqrt(n)
  t_stat = (avg - 0) / se
  
  df.loc[x, 'E[R]'] =f"{avg:.2f} ({t_stat:.2f})"

st_rev_A=df

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

-1.5964403254471802
-9.058013013460448
-0.8545505195483999
-5.644287039486463


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
st_mom_A


,E[R],alpha_FF6,alpha_q
Small Cap,-0.13 (-0.51),-0.33 (-1.12),-0.31 (-0.84)
Large Cap,0.69 (2.99),0.55 (2.48),0.69 (2.51)


In [ ]:
st_rev_A

,E[R],alpha_FF6,alpha_q
Small Cap,-1.45 (-8.52),-1.60 (-9.06),-1.66 (-7.71)
Large Cap,-0.75 (-4.20),-0.85 (-5.64),-0.82 (-4.73)


# Panel B

Indpendent Size Sorts

In [ ]:
crsp=pd.read_csv(filepath + "cleaned_crsp.csv" )
crsp['date']=pd.to_datetime(crsp['date'])
crsp.drop(['lagged_return_portfolio', 'lagged_turnover_portfolio'], axis=1, inplace=True)
#shift back mkt_cap
crsp['mkt_cap_t-1']=crsp.groupby('permno')['mkt_cap'].shift(1)
crsp.drop(['Unnamed: 0', 'level_0'], axis=1, inplace=True)

In [ ]:
#Independent NYSE Sorts
nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date']).apply(lambda x: x.set_index('permno')['mkt_cap_t-1'].quantile([0.2, 0.4, 0.6, 0.8 ]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.2: 'Q1', 0.4: 'Q2', 0.6: 'Q3', 0.8:'Q4'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q1, nyse.Q2, nyse.Q3, nyse.Q4
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date
        '''
crsp_nyse_size_bp=pd.read_sql(query, conn)

crsp_nyse_size_bp.dropna(inplace=True)
crsp=crsp_nyse_size_bp
crsp['size_portfolio'] = np.where(crsp['mkt_cap_t-1'] < crsp['Q1'], 'Microcap', 
                                   np.where(crsp['mkt_cap_t-1'] < crsp['Q2'], '2',
                                            np.where(crsp['mkt_cap_t-1'] < crsp['Q3'], '3',
                                                     np.where(crsp['mkt_cap_t-1'] < crsp['Q4'], '4', 'Megacap'))))

crsp.drop(["Q1", "Q2", "Q3", "Q4"], axis=1, inplace=True )

Conditional Lagged Return Sorts

In [ ]:
crsp.drop('level_0', axis=1, inplace=True)

nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date', 'size_portfolio']).apply(lambda x: x.set_index('permno')['return_t-1'].quantile([0.2, 0.8]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.2: 'Q2', 0.8: 'Q8'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q2, nyse.Q8
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date AND crsp.size_portfolio=nyse.size_portfolio
        '''
crsp_nyse_ret_bp=pd.read_sql(query, conn)

crsp_nyse_ret_bp.dropna(inplace=True)

crsp=crsp_nyse_ret_bp

crsp['lagged_return_portfolio'] = np.where(crsp['return_t-1'] < crsp['Q2'], 'Low Return', 
                                            np.where((crsp['Q2'] <= crsp['return_t-1']) & 
                                                     (crsp['return_t-1'] <= crsp['Q8']), 'Medium Return', 
                                                     'High Return'))
crsp.drop(['Q2', 'Q8'], axis=1, inplace=True)

Conditional Turnover Sorts

In [ ]:
crsp.drop('level_0', axis=1, inplace=True)

nyse_monthly = crsp[(crsp['exchange'] == 'NYSE')].groupby(['date', 'size_portfolio', 'lagged_return_portfolio']).apply(lambda x: x.set_index('permno')['to_t-1'].quantile([0.2, 0.8]))
nyse_monthly.reset_index(inplace=True)
nyse_monthly.rename(columns={0.2: 'Q2', 0.8: 'Q8'}, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
nyse_monthly.to_sql('nyse', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, nyse.Q2, nyse.Q8
        FROM crsp
        LEFT JOIN nyse
        ON crsp.date=nyse.date AND crsp.size_portfolio=nyse.size_portfolio AND crsp.lagged_return_portfolio=nyse.lagged_return_portfolio
        '''
crsp_nyse_to_bp=pd.read_sql(query, conn)

crsp=crsp_nyse_to_bp

crsp['lagged_turnover_portfolio'] = np.where(crsp['to_t-1'] < crsp['Q2'], 'Low Turnover', 
                                            np.where((crsp['Q2'] <= crsp['to_t-1']) & 
                                                     (crsp['to_t-1'] <= crsp['Q8']), 'Medium Turnover', 
                                                     'High Turnover'))

crsp.drop(['Q2', 'Q8'], axis=1, inplace=True)

In [ ]:
crsp.to_csv(filepath+"crsp_nyse_quintile_sorts.csv")

In [ ]:
crsp['ret_adj'] = crsp['ret_adj'].clip(lower=crsp['ret_adj'].quantile(0.01), upper=crsp['ret_adj'].quantile(0.99))

portfolio_returns = crsp.groupby(['date', 'size_portfolio', 'lagged_return_portfolio', 'lagged_turnover_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret_adj']), weights=pd.to_numeric(x['mkt_equity']))).reset_index(name='weighted_returns')
portfolio_returns.sort_values(by=['date','size_portfolio', 'lagged_return_portfolio', 'lagged_turnover_portfolio'], inplace=True)

In [ ]:
crsp

,level_0,date,index,permno,ret,shrout,altprc,exchcd,siccd,vol,...,reference_date,month,mkt_equity,to,return_t-1,to_t-1,mkt_cap_t-1,size_portfolio,lagged_return_portfolio,lagged_turnover_portfolio
0,0,1963-06-28 00:00:00,1790611,35887,-10.1407,854.0,14.62500,2.0,3599.0,114.0,...,1963-06-01 00:00:00,6,12.489750,0.133489,-10.1407,0.133489,12.489750,Microcap,Low Return,Low Turnover
1,1,1963-06-28 00:00:00,1783415,35617,1.5385,1074.0,13.87500,2.0,4925.0,52.0,...,1963-06-01 00:00:00,6,14.901750,0.048417,1.5385,0.048417,14.901750,Microcap,High Return,Low Turnover
2,2,1963-06-28 00:00:00,1509141,27326,9.6654,1629.0,72.75000,1.0,3569.0,152.0,...,1963-06-01 00:00:00,6,118.509750,0.093309,9.6654,0.093309,118.509750,3,High Return,Medium Turnover
3,3,1963-06-28 00:00:00,1602408,29946,-5.8122,2491.0,24.25000,2.0,2085.0,104.0,...,1963-06-01 00:00:00,6,60.406750,0.041750,-5.8122,0.041750,60.406750,2,Low Return,Low Turnover
4,4,1963-06-28 00:00:00,936463,17777,-12.6352,1197.0,24.75000,1.0,2011.0,387.0,...,1963-06-01 00:00:00,6,29.625750,0.323308,-12.6352,0.323308,29.625750,Microcap,Low Return,Medium Turnover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230311,2230311,2018-12-31 00:00:00,4744906,91977,-3.1157,65933.0,28.91000,1.0,2033.0,145492.0,...,2018-06-01 00:00:00,12,1991.130700,2.206664,16.4746,2.231599,1999.747890,3,High Return,Medium Turnover
2230312,2230312,2018-12-31 00:00:00,485124,13326,-18.3455,65372.0,14.51000,3.0,9999.0,134460.0,...,2018-06-01 00:00:00,12,1501.355520,2.056844,26.2971,3.270953,1158.461840,2,High Return,High Turnover
2230313,2230313,2018-12-31 00:00:00,4530303,90175,-24.8089,126947.0,57.04000,1.0,4731.0,1524301.0,...,2018-06-01 00:00:00,12,12086.216100,12.007381,-15.1264,3.442114,9630.199420,4,Low Return,Medium Turnover
2230314,2230314,2018-12-31 00:00:00,2520075,58421,-12.7475,7009.0,7.05000,2.0,3312.0,2664.0,...,2018-06-01 00:00:00,12,56.772900,0.380083,-8.1818,0.792695,56.632720,Microcap,Medium Return,Medium Turnover


In [ ]:
g = portfolio_returns[portfolio_returns['size_portfolio']=='Megacap']
g

,date,size_portfolio,lagged_return_portfolio,lagged_turnover_portfolio,weighted_returns
10,1963-06-28 00:00:00,Megacap,High Return,Medium Turnover,4.507700
11,1963-06-28 00:00:00,Megacap,Low Return,Medium Turnover,-2.290100
12,1963-06-28 00:00:00,Megacap,Medium Return,High Turnover,1.256300
13,1963-06-28 00:00:00,Megacap,Medium Return,Low Turnover,-0.613000
49,1963-07-31 00:00:00,Megacap,High Return,High Turnover,-1.736583
...,...,...,...,...,...
29978,2018-12-31 00:00:00,Megacap,Low Return,Low Turnover,-7.621828
29979,2018-12-31 00:00:00,Megacap,Low Return,Medium Turnover,-10.040534
29980,2018-12-31 00:00:00,Megacap,Medium Return,High Turnover,-10.038005
29981,2018-12-31 00:00:00,Megacap,Medium Return,Low Turnover,-9.089925


In [ ]:
#ST MOMENTUM
df = pd.DataFrame(columns=['E[R]', 'alpha_FF6', 'alpha_q'])
df.loc['Microcap'] = [None, None, None]
df.loc['2'] = [None, None, None]
df.loc['3'] = [None, None, None]
df.loc['4'] = [None, None, None]
df.loc['Megacap'] = [None, None, None]

for x in ['Microcap', '2', '3', '4', 'Megacap']:
  portfolio_returns['date']=pd.to_datetime(portfolio_returns['date'])
  portfolio_returns['month_year']=portfolio_returns['date'].dt.to_period("M")
  portfolio_returns['month_year']=portfolio_returns['month_year'].apply(str)

  portfolio_returns_size=portfolio_returns[portfolio_returns['size_portfolio']==x]

  portfolio_returns_temp1=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='High Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="Low Return")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='High Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="High Return")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)


  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, ff, on='month_year')

  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)
  print(portfolio_returns_temp)
  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_FF6'] =f"{intercept:.2f} ({tvalue:.2f})"

  print(intercept)
  print(tvalue)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, q, on='month_year')
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]
  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_q'] =f"{intercept:.2f} ({tvalue:.2f})"

  

  avg=portfolio_returns_temp['high-low'].mean()
  s = portfolio_returns_temp['high-low'].std(ddof=1)
  n = len(portfolio_returns_temp)
  se = s / np.sqrt(n)
  t_stat = (avg - 0) / se
  
  df.loc[x, 'E[R]'] =f"{avg:.2f} ({t_stat:.2f})"

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31         Microcap                Low Return   
1   1963-08-30         Microcap                Low Return   
2   1963-09-30         Microcap                Low Return   
3   1963-10-31         Microcap                Low Return   
4   1963-11-29         Microcap                Low Return   
..         ...              ...                       ...   
661 2018-08-31         Microcap                Low Return   
662 2018-09-28         Microcap                Low Return   
663 2018-10-31         Microcap                Low Return   
664 2018-11-30         Microcap                Low Return   
665 2018-12-31         Microcap                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                 High Turnover     0.679118      1963-07         Microcap   
1                 High Turnover     4.665200      1963-08         Microcap   
2                 High Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31                2                Low Return   
1   1963-08-30                2                Low Return   
2   1963-09-30                2                Low Return   
3   1963-10-31                2                Low Return   
4   1963-11-29                2                Low Return   
..         ...              ...                       ...   
661 2018-08-31                2                Low Return   
662 2018-09-28                2                Low Return   
663 2018-10-31                2                Low Return   
664 2018-11-30                2                Low Return   
665 2018-12-31                2                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                 High Turnover    -4.387665      1963-07                2   
1                 High Turnover     7.806783      1963-08                2   
2                 High Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31                3                Low Return   
1   1963-08-30                3                Low Return   
2   1963-09-30                3                Low Return   
3   1963-10-31                3                Low Return   
4   1963-11-29                3                Low Return   
..         ...              ...                       ...   
661 2018-08-31                3                Low Return   
662 2018-09-28                3                Low Return   
663 2018-10-31                3                Low Return   
664 2018-11-30                3                Low Return   
665 2018-12-31                3                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                 High Turnover    -1.786656      1963-07                3   
1                 High Turnover     1.797187      1963-08                3   
2                 High Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31                4                Low Return   
1   1963-08-30                4                Low Return   
2   1963-09-30                4                Low Return   
3   1963-10-31                4                Low Return   
4   1963-11-29                4                Low Return   
..         ...              ...                       ...   
661 2018-08-31                4                Low Return   
662 2018-09-28                4                Low Return   
663 2018-10-31                4                Low Return   
664 2018-11-30                4                Low Return   
665 2018-12-31                4                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                 High Turnover    -1.861995      1963-07                4   
1                 High Turnover     8.832965      1963-08                4   
2                 High Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
st_mom_B=df
st_mom_B

,E[R],alpha_FF6,alpha_q
Microcap,-0.58 (-2.29),-0.63 (-2.16),-0.71 (-2.19)
2,-0.12 (-0.48),-0.17 (-0.67),-0.17 (-0.56)
3,0.48 (1.86),0.42 (1.56),0.61 (2.04)
4,0.29 (1.10),0.20 (0.78),0.31 (1.03)
Megacap,0.68 (2.86),0.58 (2.56),0.68 (2.45)


In [ ]:
#ST REV
df = pd.DataFrame(columns=['E[R]', 'alpha_FF6', 'alpha_q'])
df.loc['Microcap'] = [None, None, None]
df.loc['2'] = [None, None, None]
df.loc['3'] = [None, None, None]
df.loc['4'] = [None, None, None]
df.loc['Megacap'] = [None, None, None]

for x in ['Microcap', '2', '3', '4', 'Megacap']:
  portfolio_returns['date']=pd.to_datetime(portfolio_returns['date'])
  portfolio_returns['month_year']=portfolio_returns['date'].dt.to_period("M")
  portfolio_returns['month_year']=portfolio_returns['month_year'].apply(str)

  portfolio_returns_size=portfolio_returns[portfolio_returns['size_portfolio']==x]

  portfolio_returns_temp1=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='Low Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="Low Return")]
  portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

  portfolio_returns_temp10=portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio']=='Low Turnover') & (portfolio_returns_size['lagged_return_portfolio']=="High Return")]
  portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)


  portfolio_returns_temp=pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
  portfolio_returns_temp['high-low']=portfolio_returns_temp['high_returns']-portfolio_returns_temp['low_returns']

  portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, ff, on='month_year')

  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',  'Mom   ']]

  # Add a constant term to the independent variables
  X = sm.add_constant(X)
  print(portfolio_returns_temp)
  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_FF6'] =f"{intercept:.2f} ({tvalue:.2f})"

  print(intercept)
  print(tvalue)
  portfolio_returns_temp=pd.merge(portfolio_returns_temp, q, on='month_year')
  y = portfolio_returns_temp['high-low']
  X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]
  # Add a constant term to the independent variables
  X = sm.add_constant(X)

  # Create a time series model with Newey-West standard errors
  model = sm.OLS(y, X)
  results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
  intercept = results.params['const']
  tvalue = results.tvalues['const']
  df.loc[x, 'alpha_q'] =f"{intercept:.2f} ({tvalue:.2f})"

  

  avg=portfolio_returns_temp['high-low'].mean()
  s = portfolio_returns_temp['high-low'].std(ddof=1)
  n = len(portfolio_returns_temp)
  se = s / np.sqrt(n)
  t_stat = (avg - 0) / se
  
  df.loc[x, 'E[R]'] =f"{avg:.2f} ({t_stat:.2f})"

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31         Microcap                Low Return   
1   1963-08-30         Microcap                Low Return   
2   1963-09-30         Microcap                Low Return   
3   1963-10-31         Microcap                Low Return   
4   1963-11-29         Microcap                Low Return   
..         ...              ...                       ...   
661 2018-08-31         Microcap                Low Return   
662 2018-09-28         Microcap                Low Return   
663 2018-10-31         Microcap                Low Return   
664 2018-11-30         Microcap                Low Return   
665 2018-12-31         Microcap                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                  Low Turnover     1.810477      1963-07         Microcap   
1                  Low Turnover     5.016656      1963-08         Microcap   
2                  Low Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31                2                Low Return   
1   1963-08-30                2                Low Return   
2   1963-09-30                2                Low Return   
3   1963-10-31                2                Low Return   
4   1963-11-29                2                Low Return   
..         ...              ...                       ...   
661 2018-08-31                2                Low Return   
662 2018-09-28                2                Low Return   
663 2018-10-31                2                Low Return   
664 2018-11-30                2                Low Return   
665 2018-12-31                2                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                  Low Turnover    -2.456389      1963-07                2   
1                  Low Turnover     5.548863      1963-08                2   
2                  Low Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31                3                Low Return   
1   1963-08-30                3                Low Return   
2   1963-09-30                3                Low Return   
3   1963-10-31                3                Low Return   
4   1963-11-29                3                Low Return   
..         ...              ...                       ...   
661 2018-08-31                3                Low Return   
662 2018-09-28                3                Low Return   
663 2018-10-31                3                Low Return   
664 2018-11-30                3                Low Return   
665 2018-12-31                3                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                  Low Turnover    -1.852602      1963-07                3   
1                  Low Turnover     6.457972      1963-08                3   
2                  Low Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


          date size_portfolio_x lagged_return_portfolio_x  \
0   1963-07-31                4                Low Return   
1   1963-08-30                4                Low Return   
2   1963-09-30                4                Low Return   
3   1963-10-31                4                Low Return   
4   1963-11-29                4                Low Return   
..         ...              ...                       ...   
661 2018-08-31                4                Low Return   
662 2018-09-28                4                Low Return   
663 2018-10-31                4                Low Return   
664 2018-11-30                4                Low Return   
665 2018-12-31                4                Low Return   

    lagged_turnover_portfolio_x  low_returns month_year_x size_portfolio_y  \
0                  Low Turnover     0.237831      1963-07                4   
1                  Low Turnover     5.338350      1963-08                4   
2                  Low Turnover  

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
st_rev_B=df
st_rev_B

,E[R],alpha_FF6,alpha_q
Microcap,-2.10 (-11.75),-2.23 (-11.76),-2.24 (-9.90)
2,-1.01 (-5.80),-1.00 (-5.78),-1.10 (-5.90)
3,-1.01 (-5.46),-1.15 (-6.06),-1.12 (-5.07)
4,-1.07 (-6.26),-1.11 (-6.61),-0.97 (-4.73)
Megacap,-0.68 (-3.65),-0.71 (-3.95),-0.64 (-3.23)


# Panel C

In [ ]:
crsp=pd.read_csv(filepath + "cleaned_crsp.csv" )
crsp['date']=pd.to_datetime(crsp['date'])
crsp.drop(['lagged_return_portfolio', 'lagged_turnover_portfolio'], axis=1, inplace=True)
#shift back mkt_cap
crsp['mkt_cap_t-1']=crsp.groupby('permno')['mkt_cap'].shift(1)
crsp.drop(['Unnamed: 0', 'level_0'], axis=1, inplace=True)

Rank stocks and keep top 500 stocks for each month

In [ ]:
crsp['rank'] = crsp.groupby('month')['mkt_cap_t-1'].rank(ascending=False)
crsp.dropna(inplace=True)

crsp=crsp[crsp['rank']<=500]


Rank based on Lagged Return and Assign top 250 to High Portfolio, bottom 250 to Low Portfolio

In [ ]:
crsp['return_rank'] = crsp.groupby('month')['return_t-1'].rank(ascending=False)
crsp['lagged_return_portfolio']=np.where(crsp['return_rank']<=250, 'High Return', 'Low Return' )

<ipython-input-29-cf3842ca0d87>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['return_rank'] = crsp.groupby('month')['return_t-1'].rank(ascending=False)
<ipython-input-29-cf3842ca0d87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['lagged_return_portfolio']=np.where(crsp['return_rank']<=250, 'High Return', 'Low Return' )


In [ ]:
crsp['turnover_rank'] = crsp.groupby('month')['to_t-1'].rank(ascending=False)
crsp['lagged_turnover_portfolio']=np.where(crsp['turnover_rank']<=250, 'High Turnover', 'Low Turnover' )

<ipython-input-30-2950e8e90dbf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['turnover_rank'] = crsp.groupby('month')['to_t-1'].rank(ascending=False)
<ipython-input-30-2950e8e90dbf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['lagged_turnover_portfolio']=np.where(crsp['turnover_rank']<=250, 'High Turnover', 'Low Turnover' )


In [ ]:
portfolio_returns = crsp.groupby(['date','lagged_return_portfolio', 'lagged_turnover_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret_adj']), weights=pd.to_numeric(x['mkt_equity']))).reset_index(name='weighted_returns')
portfolio_returns.sort_values(by=['date', 'lagged_return_portfolio', 'lagged_turnover_portfolio'], inplace=True)

In [ ]:
portfolio_returns

,date,lagged_return_portfolio,lagged_turnover_portfolio,weighted_returns
0,1986-01-31,High Return,Low Turnover,-2.572300
1,1986-02-28,Low Return,Low Turnover,0.313500
2,1986-04-30,Low Return,Low Turnover,3.135300
3,1986-05-30,High Return,Low Turnover,-1.776000
4,1986-06-30,Low Return,Low Turnover,-3.855600
...,...,...,...,...
965,2018-11-30,Low Return,Low Turnover,6.835554
966,2018-12-31,High Return,High Turnover,-7.454253
967,2018-12-31,High Return,Low Turnover,-8.724328
968,2018-12-31,Low Return,High Turnover,-8.583961


In [ ]:
portfolio_returns['date'] = pd.to_datetime(portfolio_returns['date'])
portfolio_returns['month_year'] = portfolio_returns['date'].dt.to_period("M").apply(str)
portfolio_returns_size = portfolio_returns

portfolio_returns_temp1 = portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio'] == 'Low Turnover') & (portfolio_returns_size['lagged_return_portfolio'] == "Low Return")]
portfolio_returns_temp1.rename(columns={'weighted_returns': 'low_returns'}, inplace=True)

portfolio_returns_temp10 = portfolio_returns_size[(portfolio_returns_size['lagged_turnover_portfolio'] == 'Low Turnover') & (portfolio_returns_size['lagged_return_portfolio'] == "High Return")]
portfolio_returns_temp10.rename(columns={'weighted_returns': 'high_returns'}, inplace=True)

portfolio_returns_temp = pd.merge(portfolio_returns_temp1, portfolio_returns_temp10, on='date')
portfolio_returns_temp['high-low'] = portfolio_returns_temp['high_returns'] - portfolio_returns_temp['low_returns']
portfolio_returns_temp.rename(columns={'month_year_y': 'month_year'}, inplace=True)
portfolio_returns_temp = pd.merge(portfolio_returns_temp, ff, on='month_year')

y = portfolio_returns_temp['high-low']
X = portfolio_returns_temp[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom   ']]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create a time series model with Newey-West standard errors
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
intercept = results.params['const']
tvalue = results.tvalues['const']

print(intercept)
print(tvalue)
portfolio_returns_temp = pd.merge(portfolio_returns_temp, q, on='month_year')

y = portfolio_returns_temp['high-low']
X = portfolio_returns_temp[['R_MKT', 'R_ME', 'R_IA', 'R_ROE']]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create a time series model with Newey-West standard errors
model = sm.OLS(y, X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 3})
intercept = results.params['const']
tvalue = results.tvalues['const']
print(intercept)
print(tvalue)

avg = portfolio_returns_temp['high-low'].mean()
s = portfolio_returns_temp['high-low'].std(ddof=1)
n = len(portfolio_returns_temp)
se = s / np.sqrt(n)
t_stat = (avg - 0) / se
print(avg)
print(t_stat)


-0.5794148629547993
-1.988155337069312
-0.5788998819378639
-1.8637075091078665
-0.6328927912010713
-2.1793860270201404


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Table 4

Import cleaned CRSP data

In [ ]:
filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"
file="CRSP_COMPUSTAT_MERGED.csv"
df=pd.read_csv(filepath + file)


In [ ]:
filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"

file="CRSPMONTHLY.csv"
crsp=pd.read_csv(filepath + file)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
crsp

In [ ]:
crsp['date'] = pd.to_datetime(crsp['date'], format="%Y%m%d")


crsp=crsp[("1960-01-01"<=crsp['date']) & (crsp['date']< "2019-01-01")]
crsp_parsed=crsp[['PERMNO', 'date', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD', 'VOL']]
crsp=crsp_parsed


x=['PERMNO', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD']

for i in x:
  crsp[i]=pd.to_numeric(crsp[i], errors='coerce')


#convert returns to percent:
crsp['RET']*=100
crsp['DLRET']*=100


#add market cap column
crsp['mkt_cap']=abs(crsp['SHROUT'] * crsp['ALTPRC'])/1000
crsp['mkt_cap'].replace(0, np.NaN, inplace=True)


#ONLY KEEP US BASED COMMON STOCKS
crsp=crsp[(crsp['SHRCD']==10)|(crsp['SHRCD']==11)]

#rename variables to lower case
crsp= crsp.rename(columns=str.lower)

#filter for relevant exchanges
exchange_mapping = {
    1: 'NYSE', 31: 'NYSE',
    2: 'AMEX', 32: 'AMEX',
    3: 'NASDAQ', 33: 'NASDAQ'
}
#create dictionary to map values of the 'exchcd' column



crsp['exchange'] = np.select(
    [crsp['exchcd'].isin(exchange_mapping.keys()), crsp['exchcd'].notnull()],
    [crsp['exchcd'].map(exchange_mapping), "Other"],
    default='Other')

#only keep NYSE, AMEX and NASDAQ stocks
crsp=crsp[(crsp['exchange'] == "NYSE") | (crsp['exchange'] == "AMEX") | (crsp['exchange'] == "NASDAQ")  ]


#Adjust Returns
crsp['ret_adj'] = crsp['ret']

mask1 = pd.isnull(crsp['dlstcd'])
crsp.loc[mask1, 'ret_adj'] = crsp.loc[mask1, 'ret']

mask2 = pd.notnull(crsp['dlstcd']) & pd.notnull(crsp['dlret'])
crsp.loc[mask2, 'ret_adj'] = crsp.loc[mask2, 'dlret']

mask3 = ((551 <= crsp['dlstcd']) & (crsp['dlstcd'] <= 574)) | (crsp['dlstcd'].isin([500, 520, 580, 584]))
crsp.loc[mask3, 'ret_adj'] = -30

crsp.loc[~(mask1 | mask2 | mask3), 'ret_adj'] = -100

crsp.drop(['shrcd', 'dlret', 'dlstcd'], inplace=True, axis=1)

#exclude financial firms 
crsp=crsp[(crsp['siccd']<6000) |(crsp['siccd']>6999) ]

#exclude stocks not in compustat

crsp_filtered=crsp[crsp['permno'].isin(df['LPERMNO'])]

#exclude stocks not in compustat

#crsp_filtered=crsp[crsp['permno'].isin(df['LPERMNO'])]

crsp['vol_adjusted'] = np.where((crsp['exchange'] == 'NASDAQ') & (crsp['date'] < '2001-02-01'), crsp['vol'] / 2,
                                 np.where((crsp['exchange'] == 'NASDAQ') & (crsp['date'] < '2002-01-01'), crsp['vol'] / 1.8,
                                          np.where((crsp['exchange'] == 'NASDAQ') & (crsp['date'] < '2004-01-01'), crsp['vol'] / 1.6,
                                                   crsp['vol'])))

crsp['date']=pd.to_datetime(crsp['date'])
crsp['year']=crsp['date'].dt.year

crsp['reference_date']=pd.to_datetime(crsp['year'].astype(str) +'-06-01')

crsp.loc[crsp['date'].dt.month < 6, 'year'] -= 1

crsp.loc[crsp['date'].dt.month < 6, 'reference_date'] = pd.to_datetime(
    crsp['year'].astype(str) + '-06-01')

crsp = crsp.drop('year', axis=1)

#extract market cap at June
crsp['month']=crsp['date'].dt.month
stock_weights=crsp[crsp['month']==6]


stock_weights.to_sql('weights', conn, if_exists='replace')
crsp.to_sql('crsp', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, weights.mkt_cap as mkt_equity
        FROM crsp
        LEFT JOIN weights ON crsp.reference_date=weights.reference_date AND crsp.permno=weights.permno
        '''

crsp_weights=pd.read_sql(query, conn)
crsp=crsp_weights

#deflate trading volume by shares outstanding:

crsp['to']=crsp['vol_adjusted']/crsp['shrout']

crsp=crsp.set_index('date')
crsp.sort_index(inplace=True)
crsp['return_t-1']=crsp.groupby('permno')['ret_adj'].shift(1)
crsp['to_t-1']=crsp.groupby('permno')['to'].shift(1)


<ipython-input-4-66b34a2409ef>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['ret_adj'] = crsp['ret']


Create prior 12-2 months return

In [ ]:
#crsp.drop(['Unnamed: 0', 'level_0'], axis=1, inplace=True)

In [ ]:
for i in range(2, 13):
  crsp[f"return_t-{i}"]=crsp.groupby('permno')['ret_adj'].shift(i)

  

In [ ]:
#cumulate columns to create lagged return column


In [ ]:
for i in range(2, 13):
  crsp[f"return_t-{i}"]=crsp[f"return_t-{i}"]/100
  crsp[f"return_t-{i}"]=crsp[f"return_t-{i}"]+1

crsp['ret_t-2-12']=(crsp['return_t-2']*crsp['return_t-3']*crsp['return_t-4']*crsp['return_t-5']*crsp['return_t-6']*crsp['return_t-7']*crsp['return_t-8']*crsp['return_t-9']*crsp['return_t-10']*crsp['return_t-11']*crsp['return_t-12'])-1



In [ ]:
#import compustat data 

In [ ]:
filepath="/content/gdrive/MyDrive/ST_MOMENTUM/"

compustat=pd.read_csv(filepath + "compustat.csv")
compustat

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,15,17,18,19,23,24,29,33,37,952,953,955,956,957,962,976,977,982) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyear,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1000,P,01,LU,25881,23369,19701113,19780630,19701231,1970.0,...,1,3089,325.0,978.0,NaN,NaN,0.0,NaN,19780630.0,NaN
1,1000,P,01,LU,25881,23369,19701113,19780630,19711231,1971.0,...,1,3089,325.0,978.0,NaN,NaN,0.0,NaN,19780630.0,NaN
2,1000,P,01,LU,25881,23369,19701113,19780630,19721231,1972.0,...,1,3089,325.0,978.0,NaN,NaN,0.0,NaN,19780630.0,NaN
3,1000,P,01,LU,25881,23369,19701113,19780630,19731231,1973.0,...,1,3089,325.0,978.0,NaN,NaN,0.0,NaN,19780630.0,NaN
4,1000,P,01,LU,25881,23369,19701113,19780630,19741231,1974.0,...,1,3089,325.0,978.0,NaN,NaN,0.0,NaN,19780630.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309500,335466,C,1,LC,14756,54928,20190418,20200531,20191231,2019.0,...,1,4400,NaN,NaN,NaN,NaN,3.0,www.hermitage-offshore.com,20210706.0,NaN
309501,339965,P,1,LC,19654,57255,20200916,E,20210131,2020.0,...,1,7370,NaN,NaN,NaN,MT,0.0,www.snowflake.com,NaN,20200910.0
309502,345556,P,1,LC,16069,55573,20201123,E,20201231,2020.0,...,1,2836,NaN,NaN,NaN,NaN,0.0,www.f-star.com,NaN,20160524.0
309503,345920,P,1,LC,20194,57591,20201210,E,20201231,2020.0,...,1,3524,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,NaN,20201210.0


In [ ]:
#parse relevant vars

x=['GVKEY', 'LPERMNO', 'datadate', 'LINKTYPE', 'LINKENDDT', 'seq', 'ceq',
                    'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv', 'pstkl', 'pstk', 'indfmt', 'datafmt',
                     'revt', 'cogs', 'xint', 'xsga']

compustat=compustat[x]

In [ ]:
dates=['datadate', 'LINKENDDT']
for date in dates:
  compustat[date]=pd.to_datetime(compustat[date], format="%Y%m%d", errors='coerce')



<ipython-input-9-eb77be84d4ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat[date]=pd.to_datetime(compustat[date], format="%Y%m%d", errors='coerce')


In [ ]:
#filter columns
compustat['datafmt']=compustat['datafmt'].apply(str)
compustat=compustat[compustat["indfmt"]=="INDL"]
compustat=compustat[compustat['datafmt']=="STD"]

#Only keep valid links
compustat=compustat[(compustat["LINKTYPE"]=="LU")| (compustat["LINKTYPE"]=="LC")]

#Only keep links active at datadate
compustat=compustat[(compustat['datadate']<=compustat["LINKENDDT"]) | pd.isnull(compustat['LINKENDDT'])]

<ipython-input-10-861083e8769c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat['datafmt']=compustat['datafmt'].apply(str)


In [ ]:
compustat.to_sql("compustat", conn, if_exists='replace')

query='''
        SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate, at,
        COALESCE(seq, ceq+pstk, at-lt) + COALESCE(txditc, txdb + itcb, 0)- COALESCE(pstkrv, pstkl, pstk, 0) as book_value, (revt-cogs-xint-xsga) / COALESCE(seq, ceq+pstk, at-lt) + COALESCE(txditc, txdb, + itcb, 0) -COALESCE(pstkrv, pstkl, pstk, 0) as operating_profitability
        FROM compustat'''

compustat_calc=pd.read_sql(query, conn)

In [ ]:
compustat=compustat_calc

In [ ]:
compustat.sort_values(['permno', 'datadate'], inplace=True)
compustat['asset_t-1']=compustat.groupby('permno')['at'].shift(1)

compustat['asset_growth']=compustat['at']/compustat['asset_t-1']
compustat['da/a_t-1']=compustat['asset_growth']/compustat['asset_t-1']

In [ ]:
#add reference date for matching
compustat['datadate']=pd.to_datetime(compustat['datadate'], errors='coerce')

compustat['year']=compustat['datadate'].dt.year
compustat['year']=pd.to_numeric(compustat['year'], errors='coerce')
compustat['reference_date']=compustat['year']+1
compustat['reference_date']=compustat['reference_date'].apply(str)
compustat['reference_date']+="-06-01"
compustat['reference_date']=pd.to_datetime(compustat['reference_date'])


In [ ]:
compustat

,gvkey,permno,datadate,at,book_value,operating_profitability,asset_t-1,asset_growth,da/a_t-1,year,reference_date
157967,13007,10000,1986-10-31,2.115,0.418,-1.562201,NaN,NaN,NaN,1986,1987-06-01
157872,12994,10001,1986-06-30,12.242,7.037,NaN,NaN,NaN,NaN,1986,1987-06-01
157873,12994,10001,1987-06-30,11.771,7.038,NaN,12.242,0.961526,0.078543,1987,1988-06-01
157874,12994,10001,1988-06-30,11.735,7.286,NaN,11.771,0.996942,0.084695,1988,1989-06-01
157875,12994,10001,1989-06-30,18.565,8.466,NaN,11.735,1.582020,0.134812,1989,1990-06-01
...,...,...,...,...,...,...,...,...,...,...,...
303694,184996,93436,2016-12-31,22664.076,4761.695,0.011752,8092.460,2.800641,0.000346,2016,2017-06-01
303695,184996,93436,2017-12-31,28655.372,4237.312,-0.123959,22664.076,1.264352,0.000056,2017,2018-06-01
303696,184996,93436,2018-12-31,29739.614,4923.243,0.186254,28655.372,1.037837,0.000036,2018,2019-06-01
303697,184996,93436,2019-12-31,34309.000,6618.000,0.222273,29739.614,1.153646,0.000039,2019,2020-06-01


Merge compustat data into stocks data

In [ ]:
#compustat.rename(columns={'da_a_t-1': 'da_a_t_1'}, inplace=True)
crsp.to_sql("crsp", conn, if_exists='replace')
compustat.to_sql("compustat", conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, compustat.gvkey, compustat.datadate, compustat.book_value, compustat.operating_profitability, compustat.asset_growth, compustat.da_a_t_1
        FROM crsp
        LEFT JOIN compustat
        ON crsp.permno=compustat.permno
        AND crsp.reference_date=compustat.reference_date
        '''
crsp_comp=pd.read_sql(query, conn)

In [ ]:
crsp_comp

,date,index,permno,ret,shrout,altprc,exchcd,siccd,vol,mkt_cap,...,return_t-10,return_t-11,return_t-12,ret_t-2-12,gvkey,datadate,book_value,operating_profitability,asset_growth,da_a_t_1
0,1960-01-29 00:00:00,694518,15157,-0.7246,505.0,34.25000,1.0,2250.0,32.0,17.296250,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
1,1960-01-29 00:00:00,801076,16272,-12.9670,3022.0,49.50000,1.0,3250.0,195.0,149.589000,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
2,1960-01-29 00:00:00,1305875,22939,-3.3493,4858.0,25.25000,1.0,5340.0,626.0,122.664500,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
3,1960-01-29 00:00:00,955004,17961,-11.1554,1799.0,55.75000,1.0,3510.0,105.0,100.294250,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
4,1960-01-29 00:00:00,426899,12802,-4.6995,857.0,21.50000,1.0,3010.0,134.0,18.425500,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666223,2018-12-31 00:00:00,670580,14925,14.1479,20758.0,3.55000,3.0,4822.0,13133.0,73.690900,...,0.882184,0.887755,0.963145,-0.200246,153957.0,2017-12-31 00:00:00,5.880,NaN,1.061025,0.098553
2666224,2018-12-31 00:00:00,1525223,27684,-17.1611,108951.0,36.10000,3.0,5065.0,247435.0,3933.131100,...,1.004706,1.072691,0.961120,-0.014833,1919.0,2017-06-30 00:00:00,5182.068,0.124947,0.862968,0.000077
2666225,2018-12-31 00:00:00,4051820,85424,-23.5099,35777.0,30.03000,1.0,3533.0,127498.0,1074.383310,...,0.872217,1.082809,0.994786,-0.112409,65671.0,2017-12-31 00:00:00,1297.893,3.458933,0.957849,0.000655
2666226,2018-12-31 00:00:00,671374,14934,-30.1848,25228.0,0.68000,3.0,7381.0,38776.0,17.155040,...,1.138249,0.620000,2.651515,-0.166666,18490.0,2017-12-31 00:00:00,19.465,-1.945299,1.363066,0.038512


In [ ]:
crsp_comp

,date,index,permno,ret,shrout,altprc,exchcd,siccd,vol,mkt_cap,...,return_t-10,return_t-11,return_t-12,ret_t-2-12,gvkey,datadate,book_value,operating_profitability,asset_growth,da_a_t_1
0,1960-01-29 00:00:00,694518,15157,-0.7246,505.0,34.25000,1.0,2250.0,32.0,17.296250,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
1,1960-01-29 00:00:00,801076,16272,-12.9670,3022.0,49.50000,1.0,3250.0,195.0,149.589000,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
2,1960-01-29 00:00:00,1305875,22939,-3.3493,4858.0,25.25000,1.0,5340.0,626.0,122.664500,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
3,1960-01-29 00:00:00,955004,17961,-11.1554,1799.0,55.75000,1.0,3510.0,105.0,100.294250,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
4,1960-01-29 00:00:00,426899,12802,-4.6995,857.0,21.50000,1.0,3010.0,134.0,18.425500,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666223,2018-12-31 00:00:00,670580,14925,14.1479,20758.0,3.55000,3.0,4822.0,13133.0,73.690900,...,0.882184,0.887755,0.963145,-0.200246,153957.0,2017-12-31 00:00:00,5.880,NaN,1.061025,0.098553
2666224,2018-12-31 00:00:00,1525223,27684,-17.1611,108951.0,36.10000,3.0,5065.0,247435.0,3933.131100,...,1.004706,1.072691,0.961120,-0.014833,1919.0,2017-06-30 00:00:00,5182.068,0.124947,0.862968,0.000077
2666225,2018-12-31 00:00:00,4051820,85424,-23.5099,35777.0,30.03000,1.0,3533.0,127498.0,1074.383310,...,0.872217,1.082809,0.994786,-0.112409,65671.0,2017-12-31 00:00:00,1297.893,3.458933,0.957849,0.000655
2666226,2018-12-31 00:00:00,671374,14934,-30.1848,25228.0,0.68000,3.0,7381.0,38776.0,17.155040,...,1.138249,0.620000,2.651515,-0.166666,18490.0,2017-12-31 00:00:00,19.465,-1.945299,1.363066,0.038512


In [ ]:
# Calculate the 1st and 99th percentile of each variable

q1_returns = crsp_comp['return_t-1'].quantile(0.01)
q99_returns = crsp_comp['return_t-1'].quantile(0.99)
q1_variable1 = crsp_comp['to_t-1'].quantile(0.01)
q99_variable1 = crsp_comp['to_t-1'].quantile(0.99)

# Trim the variables at the 1st and 99th percentile
crsp_comp['return_t-1'] = crsp_comp['return_t-1'].clip(lower=q1_returns, upper=q99_returns)
crsp_comp['to_t-1'] = crsp_comp['to_t-1'].clip(lower=q1_variable1, upper=q99_variable1)

# Calculate the average and standard deviation of each variable across all stocks for each month
grouped = crsp_comp.groupby('date')
mean_returns = grouped['return_t-1'].mean()
std_returns = grouped['return_t-1'].std()
mean_variable1 = grouped['to_t-1'].mean()
std_variable1 = grouped['to_t-1'].std()

# Create new columns for the standardized variables
crsp_comp['return_t-1_standardized'] = (crsp_comp['return_t-1'] - mean_returns.loc[crsp_comp['date']].values) / std_returns.loc[crsp_comp['date']].values
crsp_comp['to_t-1_standardized'] = (crsp_comp['to_t-1'] - mean_variable1.loc[crsp_comp['date']].values) / std_variable1.loc[crsp_comp['date']].values


In [ ]:
crsp_comp

,date,index,permno,ret,shrout,altprc,exchcd,siccd,vol,mkt_cap,...,return_t-12,ret_t-2-12,gvkey,datadate,book_value,operating_profitability,asset_growth,da_a_t_1,return_t-1_standardized,to_t-1_standardized
0,1960-01-29 00:00:00,694518,15157,-0.7246,505.0,34.25000,1.0,2250.0,32.0,17.296250,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,1960-01-29 00:00:00,801076,16272,-12.9670,3022.0,49.50000,1.0,3250.0,195.0,149.589000,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
2,1960-01-29 00:00:00,1305875,22939,-3.3493,4858.0,25.25000,1.0,5340.0,626.0,122.664500,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
3,1960-01-29 00:00:00,955004,17961,-11.1554,1799.0,55.75000,1.0,3510.0,105.0,100.294250,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
4,1960-01-29 00:00:00,426899,12802,-4.6995,857.0,21.50000,1.0,3010.0,134.0,18.425500,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666223,2018-12-31 00:00:00,670580,14925,14.1479,20758.0,3.55000,3.0,4822.0,13133.0,73.690900,...,0.963145,-0.200246,153957.0,2017-12-31 00:00:00,5.880,NaN,1.061025,0.098553,-0.256278,-0.937186
2666224,2018-12-31 00:00:00,1525223,27684,-17.1611,108951.0,36.10000,3.0,5065.0,247435.0,3933.131100,...,0.961120,-0.014833,1919.0,2017-06-30 00:00:00,5182.068,0.124947,0.862968,0.000077,0.656140,0.453576
2666225,2018-12-31 00:00:00,4051820,85424,-23.5099,35777.0,30.03000,1.0,3533.0,127498.0,1074.383310,...,0.994786,-0.112409,65671.0,2017-12-31 00:00:00,1297.893,3.458933,0.957849,0.000655,-0.474195,0.046979
2666226,2018-12-31 00:00:00,671374,14934,-30.1848,25228.0,0.68000,3.0,7381.0,38776.0,17.155040,...,2.651515,-0.166666,18490.0,2017-12-31 00:00:00,19.465,-1.945299,1.363066,0.038512,-0.718640,-0.140556


In [ ]:
crsp_comp=crsp_comp[("1963-07-01"<=crsp_comp['date']) & (crsp_comp['date']< "2019-01-01")]



In [ ]:
crsp_comp

,date,index,permno,ret,shrout,altprc,exchcd,siccd,vol,mkt_cap,...,return_t-12,ret_t-2-12,gvkey,datadate,book_value,operating_profitability,asset_growth,da_a_t_1,return_t-1_standardized,to_t-1_standardized
54940,1963-07-31 00:00:00,574226,14154,-13.2979,1400.0,-10.18750,1.0,4131.0,174.0,14.262500,...,1.018519,0.870372,NaN,None,NaN,NaN,NaN,NaN,-0.752154,-0.051402
54941,1963-07-31 00:00:00,1067246,19289,-7.8125,987.0,29.50000,1.0,3547.0,102.0,29.116500,...,1.031022,0.046910,7280.0,1962-12-31 00:00:00,37.436,0.138102,0.947977,0.018265,-0.785893,-0.195982
54942,1963-07-31 00:00:00,942371,17830,-3.8043,6427.0,44.25000,1.0,3724.0,680.0,284.394750,...,1.046512,0.135514,10983.0,1962-12-31 00:00:00,257.480,-32.608788,1.068322,0.002122,-0.103750,-0.263791
54943,1963-07-31 00:00:00,705210,15253,NaN,1082.0,2.25000,1.0,3589.0,285.0,2.434500,...,0.931818,-0.022726,NaN,None,NaN,NaN,NaN,NaN,-4.985048,3.103876
54944,1963-07-31 00:00:00,1681512,32395,9.4017,560.0,8.00000,2.0,5040.0,62.0,4.480000,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,0.588509,-0.183099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666223,2018-12-31 00:00:00,670580,14925,14.1479,20758.0,3.55000,3.0,4822.0,13133.0,73.690900,...,0.963145,-0.200246,153957.0,2017-12-31 00:00:00,5.880,NaN,1.061025,0.098553,-0.256278,-0.937186
2666224,2018-12-31 00:00:00,1525223,27684,-17.1611,108951.0,36.10000,3.0,5065.0,247435.0,3933.131100,...,0.961120,-0.014833,1919.0,2017-06-30 00:00:00,5182.068,0.124947,0.862968,0.000077,0.656140,0.453576
2666225,2018-12-31 00:00:00,4051820,85424,-23.5099,35777.0,30.03000,1.0,3533.0,127498.0,1074.383310,...,0.994786,-0.112409,65671.0,2017-12-31 00:00:00,1297.893,3.458933,0.957849,0.000655,-0.474195,0.046979
2666226,2018-12-31 00:00:00,671374,14934,-30.1848,25228.0,0.68000,3.0,7381.0,38776.0,17.155040,...,2.651515,-0.166666,18490.0,2017-12-31 00:00:00,19.465,-1.945299,1.363066,0.038512,-0.718640,-0.140556


In [ ]:
crsp_comp['total_market_cap'] = crsp_comp.groupby('date')['mkt_cap'].transform('sum')
crsp_comp['weight']=crsp_comp['mkt_cap']/crsp_comp['total_market_cap']

<ipython-input-49-0845bab7f695>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_comp['total_market_cap'] = crsp_comp.groupby('date')['mkt_cap'].transform('sum')
<ipython-input-49-0845bab7f695>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_comp['weight']=crsp_comp['mkt_cap']/crsp_comp['total_market_cap']


In [ ]:
q99_returns = crsp_comp['ret_adj'].quantile(0.99)
crsp_comp['ret_adj'] = crsp_comp['ret_adj'].clip(upper=q99_returns)


crsp_comp['interact'] = crsp_comp['return_t-1_standardized'] * crsp_comp['to_t-1_standardized']
crsp_comp.dropna(inplace=True)
unique_dates=crsp_comp['date'].unique()

final_df= pd.DataFrame(columns=['date', 'ret', 'to', 'interact'])

for date in unique_dates:
  crsp_temp=crsp_comp[crsp_comp['date']==date]

  import statsmodels.api as sm

  X = crsp_temp[['return_t-1_standardized', 'to_t-1_standardized', 'interact']]
  Y = crsp_temp['ret_adj']
  weights = crsp_temp['weight']
  # Fit the regression model without a constant term
  model = sm.WLS(Y, X, weights=weights).fit()

  params=model.params
  final_df = final_df.append({'date': date, 'ret': params[0], 'to': params[1], 'interact': params[2]}, ignore_index=True)
  print(date)


<ipython-input-54-bfa89a86aa9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_comp['ret_adj'] = crsp_comp['ret_adj'].clip(upper=q99_returns)
<ipython-input-54-bfa89a86aa9e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_comp['interact'] = crsp_comp['return_t-1_standardized'] * crsp_comp['to_t-1_standardized']


1963-07-31 00:00:00
1963-08-30 00:00:00
1963-09-30 00:00:00
1963-10-31 00:00:00
1963-11-29 00:00:00
1963-12-31 00:00:00
1964-01-31 00:00:00
1964-02-28 00:00:00
1964-03-31 00:00:00
1964-04-30 00:00:00
1964-05-28 00:00:00
1964-06-30 00:00:00
1964-07-31 00:00:00
1964-08-31 00:00:00
1964-09-30 00:00:00
1964-10-30 00:00:00
1964-11-30 00:00:00
1964-12-31 00:00:00
1965-01-29 00:00:00
1965-02-26 00:00:00
1965-03-31 00:00:00
1965-04-30 00:00:00
1965-05-28 00:00:00
1965-06-30 00:00:00
1965-07-30 00:00:00
1965-08-31 00:00:00
1965-09-30 00:00:00
1965-10-29 00:00:00
1965-11-30 00:00:00
1965-12-31 00:00:00
1966-01-31 00:00:00
1966-02-28 00:00:00
1966-03-31 00:00:00
1966-04-29 00:00:00
1966-05-31 00:00:00
1966-06-30 00:00:00
1966-07-29 00:00:00
1966-08-31 00:00:00
1966-09-30 00:00:00
1966-10-31 00:00:00
1966-11-30 00:00:00
1966-12-30 00:00:00
1967-01-31 00:00:00
1967-02-28 00:00:00
1967-03-31 00:00:00
1967-04-28 00:00:00
1967-05-31 00:00:00
1967-06-30 00:00:00
1967-07-31 00:00:00
1967-08-31 00:00:00


In [ ]:
final_df.mean()

<ipython-input-52-97bbb8d6bc39>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_df.mean()


ret        -0.125355
to          0.265318
interact    0.256102
dtype: float64